In [1]:
import numpy as np
import cv2

In [2]:
cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height
while(True):
    ret, frame = cap.read()
#     frame = cv2.flip(frame, -1) # Flip camera vertically
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('frame', frame)
#     cv2.imshow('gray', gray)
    
    k = cv2.waitKey(15) & 0xff
    if k == 27: # press 'ESC' to quit
        break
cap.release()
cv2.destroyAllWindows()

## Face Detection

In [3]:
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height

while True:
    ret, image = cap.read()
    faces = faceCascade.detectMultiScale(
        image
    )
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255))
        
    cv2.imshow('video', image)
    k = cv2.waitKey(15)
    if k==27:
        break
cap.release()
cv2.destroyAllWindows()

## Mask detection

In [4]:
from tensorflow.keras.models import load_model

### VGG

In [5]:
model = load_model('./models/vgg_mask.h5')

In [ ]:
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height

while True:
    ret, image = cap.read()
    faces = faceCascade.detectMultiScale(
        image
    )
    for (x, y, w, h) in faces:
        crop = image[y:y+h, x:x+w]
        resized = cv2.resize(crop, (64, 64), interpolation=cv2.INTER_AREA)
        
        pred = np.argmax(model.predict(resized[np.newaxis, ...]/255.))
        
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255) if pred else (0, 255, 0))
        cv2.putText(image, 'no mask' if pred else 'mask', (x, y), cv2.FONT_HERSHEY_COMPLEX, 1,
                    (0, 0, 255) if pred else (0, 255, 0), 1, cv2.LINE_AA)
        
    cv2.imshow('video', image)
    k = cv2.waitKey(30)
    if k==27:
        break
cap.release()
cv2.destroyAllWindows()

### RESNET50V2

In [7]:
model = load_model('./models/resnet50v2.h5')

In [17]:
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height

while True:
    ret, image = cap.read()
    faces = faceCascade.detectMultiScale(
        image
    )
    for (x, y, w, h) in faces:
        crop = image[y:y+h, x:x+w]
        resized = cv2.resize(crop, (224, 224), interpolation=cv2.INTER_AREA)
        
        pred = np.argmax(model.predict(resized[np.newaxis, ...]))
        if pred==0:
            text='incorrect'
            clr = (255, 0, 0)
        if pred==1:
            text='mask'
            clr = (0, 255, 0)
        if pred==0:
            text='no mask'
            clr = (0, 0, 255)
        
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255) if pred else (0, 255, 0))
        cv2.putText(image, text, (x, y), cv2.FONT_HERSHEY_COMPLEX, 1,
                    clr, 1, cv2.LINE_AA)
        
    cv2.imshow('video', image)
    k = cv2.waitKey(30)
    if k==27:
        break
cap.release()
cv2.destroyAllWindows()

In [14]:
np.array([0.9, 0.1, 0.2]).argmax()

0